<a href="https://colab.research.google.com/github/YasrajDhal/Electricity-Demand-Prediction/blob/main/Energy%20consumption%20prediction%20using%20LSTM-GRU%20in%20PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy consumption prediction using LSTM/GRU in PyTorch

In this notebook, we'll be using GRU and LSTM models for a time series prediction task and we will compare the performance of the GRU model against an LSTM model as well. The dataset that we will be using is the Hourly Energy Consumption dataset which can be found on [Kaggle](https://www.kaggle.com/robikscube/hourly-energy-consumption). The dataset contains power consumption data across different regions around the United States recorded on an hourly basis.

The goal of this implementation is to **create a model that can accurately predict the energy usage in the next hour** given historical usage data. We will be using both the GRU and LSTM model to train on a set of historical data and evaluate both models on an unseen test set. To do so, we'll start with feature selection, data-preprocessing, followed by defining, training and eventually evaluating the models.

We will use the PyTorch library to implement both types of models along with other common Python libraries used in data analytics.

## GRU/LSTM cells

* Long Short-Term Memory networks (LSTMs) have great memories and can remember information which the vanilla RNN is unable to!

* The Gated Recurrent Unit (GRU) is the younger sibling of the more popular Long Short-Term Memory (LSTM) network, and also a type of Recurrent Neural Network (RNN). Just like its sibling, GRUs are able to effectively retain long-term dependencies in sequential data. And additionally, they can address the “short-term memory” issue plaguing vanilla RNNs.

## The ML Pipeline

In [9]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

In [10]:
print(torch.__version__)

2.5.1+cu124


## Exploratory Data Analysis (EDA)

In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

data_dir='/content/drive/MyDrive/electricity_data'

Mounted at /content/drive


In [12]:
pd.read_csv(os.path.join(data_dir, "DEOK_hourly.csv")).head()

,Datetime,DEOK_MW
0,2012-12-31 01:00:00,2945.0
1,2012-12-31 02:00:00,2868.0
2,2012-12-31 03:00:00,2812.0
3,2012-12-31 04:00:00,2812.0
4,2012-12-31 05:00:00,2860.0


We have a total of **12** *.csv* files containing hourly energy trend data (*'est_hourly.paruqet'* and *'pjm_hourly_est.csv'* are not used). In our next step, we will be reading these files and pre-processing these data in this order:
- Getting the time data of each individual time step and generalizing them
    - Hour of the day *i.e. 0-23*
    - Day of the week *i.e. 1-7*
    - Month *i.e. 1-12*
    - Day of the year *i.e. 1-365*
    
    
- Scale the data to values between 0 and 1
    - Algorithms tend to perform better or converge faster when features are on a relatively similar scale and/or close to normally distributed
    - Scaling preserves the shape of the original distribution and doesn't reduce the importance of outliers
    
    
- Group the data into sequences to be used as inputs to the model and store their corresponding labels
    - The **sequence length** or **window_size period** is the number of data points in history that the model will use to make the prediction
    - The label will be the next data point in time after the last one in the input sequence
    

- The inputs and labels will then be split into training and test sets

## Create training instances by moving sliding window

In [13]:
def move_sliding_window(data, window_size, inputs_cols_indices, label_col_index):
    """
    data: numpy array including data
    window_size: size of window
    inputs_cols_indices: col indices to include
    """

    # (# instances created by movement, seq_len (timestamps), # features (input_len))
    inputs = np.zeros((len(data) - window_size, window_size, len(inputs_cols_indices)))
    labels = np.zeros(len(data) - window_size)

    for i in range(window_size, len(data)):
        inputs[i - window_size] = data[i - window_size : i, inputs_cols_indices]
        labels[i - window_size] = data[i, label_col_index]
    inputs = inputs.reshape(-1, window_size, len(inputs_cols_indices))
    labels = labels.reshape(-1, 1)
    print(inputs.shape, labels.shape)

    return inputs, labels

## Integrate files to build the training set
To speed things up, I will only be using `num_files_for_dataset` .csv files for creating my dataset. Feel free to run it yourself with the entire dataset if you have the time and computing capacity.

In [14]:
label_col_index = 0  # consumption as label to predict
inputs_cols_indices = range(
    5
)  # use (consumption, hour, dayofweek, month, dayofyear) columns as features

# Define window_size period and split inputs/labels
window_size = 90

# The scaler objects will be stored in this dictionary so that our output test data from the model can be re-scaled during evaluation
label_scalers = {}

train_x = []
test_x = {}
test_y = {}

# Skipping the files we're not using
processing_files = [
    file for file in os.listdir(data_dir) if os.path.splitext(file)[1] == ".csv"
]

num_files_for_dataset = 5

for file in tqdm_notebook(processing_files[:num_files_for_dataset]):
    print(f"Processing {file} ...")
    # Store csv file in a Pandas DataFrame
    df = pd.read_csv(os.path.join(data_dir, file), parse_dates=["Datetime"])

    # Processing the time data into suitable input formats
    df["hour"] = df.apply(lambda x: x["Datetime"].hour, axis=1)
    df["dayofweek"] = df.apply(lambda x: x["Datetime"].dayofweek, axis=1)
    df["month"] = df.apply(lambda x: x["Datetime"].month, axis=1)
    df["dayofyear"] = df.apply(lambda x: x["Datetime"].dayofyear, axis=1)
    df = df.sort_values("Datetime").drop("Datetime", axis=1)

    # Scaling the input data
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    data = sc.fit_transform(df.values)

    # Obtaining the scaler for the labels(usage data) so that output can be
    # re-scaled to actual value during evaluation
    label_sc.fit(df.iloc[:, label_col_index].values.reshape(-1, 1))
    label_scalers[file] = label_sc

    # Move the window
    inputs, labels = move_sliding_window(
        data,
        window_size,
        inputs_cols_indices=inputs_cols_indices,
        label_col_index=label_col_index,
    )

    # CONCAT created instances from all .csv files.
    # Split data into train/test portions and combining all data from different files into a single array
    test_portion = int(0.1 * len(inputs))
    if len(train_x) == 0:  # first iteration
        train_x = inputs[:-test_portion]
        train_y = labels[:-test_portion]
    else:
        train_x = np.concatenate((train_x, inputs[:-test_portion]))
        train_y = np.concatenate((train_y, labels[:-test_portion]))
    test_x[file] = inputs[-test_portion:]
    test_y[file] = labels[-test_portion:]

  0%|          | 0/5 [00:00<?, ?it/s]

Processing AEP_hourly.csv ...
(121183, 90, 5) (121183, 1)
Processing FE_hourly.csv ...
(62784, 90, 5) (62784, 1)
Processing NI_hourly.csv ...
(58360, 90, 5) (58360, 1)
Processing DOM_hourly.csv ...
(116099, 90, 5) (116099, 1)
Processing DUQ_hourly.csv ...
(118978, 90, 5) (118978, 1)


![input_shape.png](https://github.com/iamirmasoud/energy_consumption_prediction/blob/master/imgs/input_shape.png?raw=1)

## What have we made?

In [15]:
train_x.shape, test_x["NI_hourly.csv"].shape

((429666, 90, 5), (5836, 90, 5))

## Pytorch data loaders/generators

To improve the speed of our training, we can process the data in batches so that the model does not need to update its weights as frequently. The `TensorDataset` and `DataLoader` classes are useful for splitting our data into batches and shuffling them.

In [16]:
batch_size = 1024

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))

# Drop the last incomplete batch
train_loader = DataLoader(
    train_data, shuffle=True, batch_size=batch_size, drop_last=True
)

In [17]:
print(
    f"Train Size: {train_x.shape}, Batch Size: {batch_size}, # of iterations per epoch: {int(train_x.shape[0]/batch_size)}"
)

Train Size: (429666, 90, 5), Batch Size: 1024, # of iterations per epoch: 419


In [18]:
# release some memory
del train_x, train_y

We can also check if we have any GPUs to speed up our training time by many folds. If you’re using "https://colab.research.google.com/" with GPU to run this code, the training time will be significantly reduced.

In [19]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")

Next, we'll be defining the structure of the GRU and LSTM models. Both models have the same structure, with the only difference being the **recurrent layer** (GRU/LSTM) and the initializing of the hidden state. The hidden state for the LSTM is a tuple containing both the **cell state** and the **hidden state**, whereas the **GRU only has a single hidden state**.
Please refer to official PyTorch documentation to get familiar with GRU and LSTM interfaces in PyTorch:

- https://pytorch.org/docs/stable/nn.html#torch.nn.GRU
- https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM

You can also detailed tutorials about Recurrent Neural Networks on my [Blog](http://www.sefidian.com/archives/) and [Github](https://github.com/iamirmasoud).

# LSTM
![lstm1.png](https://github.com/iamirmasoud/energy_consumption_prediction/blob/master/imgs/lstm1.png?raw=1)
![lstm2.png](https://github.com/iamirmasoud/energy_consumption_prediction/blob/master/imgs/lstm2.png?raw=1)

# GRU
![gru1.png](https://github.com/iamirmasoud/energy_consumption_prediction/blob/master/imgs/gru1.png?raw=1)
![gru2.png](https://github.com/iamirmasoud/energy_consumption_prediction/blob/master/imgs/gru2.png?raw=1)

In [20]:
import torch
import torch.nn as nn
import time
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.gru = nn.GRU(
            input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob
        )
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, h):
        out, h = self.gru(x, h)
        # print(out[:, -1].shape, h.shape)
        # select hidden state of last timestamp (t=90) (1024, 256)
        out = self.fc(self.relu(out[:, -1]))  # out[:, -1, :]
        # print(out.shape) # (1024, 1)
        return out, h

    def init_hidden(self, batch_size):
        # Initialze h_0 with zeros
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        )
        return hidden


class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
            input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob
        )
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:, -1]))
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        # Initialze h_0, c_0 with zeros
        hidden = (
            weight.new(self.n_layers, batch_size, self.hidden_dim)
            .zero_()
            .to(device),  # h_0
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
        )
        return hidden


class ARIMAModel:
    def __init__(self, order=(1, 1, 1)):
        """
        Initialize ARIMA model with specified order

        Parameters:
        -----------
        order : tuple
            (p, d, q) order of the ARIMA model
            p: The number of lag observations (AR order)
            d: The degree of differencing
            q: The size of the moving average window (MA order)
        """
        self.order = order
        self.model = None
        self.fitted_model = None

    def fit(self, train_data):
        """
        Fit ARIMA model to the training data

        Parameters:
        -----------
        train_data : array-like
            Time series data for training

        Returns:
        --------
        self : returns an instance of self
        """
        # Convert to pandas Series if not already
        if not isinstance(train_data, pd.Series):
            train_data = pd.Series(train_data)

        # Fit ARIMA model
        self.model = ARIMA(train_data, order=self.order)
        self.fitted_model = self.model.fit()
        return self

    def predict(self, steps=1):
        """
        Make predictions using the fitted model

        Parameters:
        -----------
        steps : int
            Number of steps to forecast ahead

        Returns:
        --------
        predictions : array
            Predicted values
        """
        if self.fitted_model is None:
            raise ValueError("Model has not been fitted yet. Call 'fit' first.")

        # Get forecast
        forecast = self.fitted_model.forecast(steps=steps)
        return forecast

    def evaluate(self, test_data):
        """
        Evaluate model performance on test data

        Parameters:
        -----------
        test_data : array-like
            Actual values to compare predictions against

        Returns:
        --------
        mse : float
            Mean squared error
        """
        if self.fitted_model is None:
            raise ValueError("Model has not been fitted yet. Call 'fit' first.")

        # Make predictions for test period
        predictions = self.predict(steps=len(test_data))

        # Calculate MSE
        mse = mean_squared_error(test_data, predictions)
        return mse

    def summary(self):
        """
        Return model summary information

        Returns:
        --------
        summary : object
            Summary of the fitted model
        """
        if self.fitted_model is None:
            raise ValueError("Model has not been fitted yet. Call 'fit' first.")

        return self.fitted_model.summary()

In [21]:
def train(
    train_loader,
    learn_rate,
    hidden_dim=256,
    n_layers=2,
    n_epochs=5,
    model_type="GRU",
    print_every=100,
):

    input_dim = next(iter(train_loader))[0].shape[2]  # 5

    # Batch generator (train_data, train_label)
    # print(next(iter(train_loader))[0].shape, next(iter(train_loader))[1].shape) # torch.Size([1024, 90, 5]) torch.Size([1024, 1])

    output_dim = 1

    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)

    # Defining loss function and optimizer
    criterion = nn.MSELoss()  # Mean Squared Error
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []

    # Start training loop
    for epoch in range(1, n_epochs + 1):
        start_time = time.process_time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.0
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            # Unpcak both h_0 and c_0
            elif model_type == "LSTM":
                h = tuple([e.data for e in h])

            # Set the gradients to zero before starting to do backpropragation because
            # PyTorch accumulates the gradients on subsequent backward passes
            model.zero_grad()

            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())

            # Perform backpropragation
            loss.backward()
            optimizer.step()

            avg_loss += loss.item()
            if counter % print_every == 0:
                print(
                    f"Epoch {epoch} - Step: {counter}/{len(train_loader)} - Average Loss for Epoch: {avg_loss/counter}"
                )
        current_time = time.process_time()

        print(
            f"Epoch {epoch}/{n_epochs} Done, Total Loss: {avg_loss/len(train_loader)}"
        )

        print(f"Time Elapsed for Epoch: {current_time-start_time} seconds")

        epoch_times.append(current_time - start_time)

    print(f"Total Training Time: {sum(epoch_times)} seconds")
    return model


def train_arima(
    train_data,
    test_data=None,
    order=(1, 1, 1),
    print_every=100
):
    """
    Train an ARIMA model on the provided data

    Parameters:
    -----------
    train_data : array-like
        Time series data for training
    test_data : array-like, optional
        Time series data for testing
    order : tuple
        (p, d, q) order of the ARIMA model
    print_every : int
        Print status update every this many samples

    Returns:
    --------
    model : ARIMAModel
        Trained ARIMA model
    """
    print("Starting Training of ARIMA model")
    start_time = time.process_time()

    # Convert data to the right format if needed
    # If train_data is a tensor or DataFrame, convert to numpy array
    if hasattr(train_data, 'numpy'):
        train_data = train_data.numpy()

    # For multivariate data, we need to extract the target variable
    # Assuming the target is the last column/feature
    if isinstance(train_data, np.ndarray) and train_data.ndim > 1:
        if train_data.shape[1] > 1:
            print("Note: ARIMA uses univariate data. Using the last column as target.")
            train_data = train_data[:, -1]

    # Create and fit the model
    model = ARIMAModel(order=order)

    try:
        print(f"Fitting ARIMA{order} model...")
        model.fit(train_data)
        print("Model fitting completed")

        # Print model summary
        print("\nModel Summary:")
        print(model.summary())

        # Evaluate on test data if provided
        if test_data is not None:
            # Prepare test data if needed
            if hasattr(test_data, 'numpy'):
                test_data = test_data.numpy()

            if isinstance(test_data, np.ndarray) and test_data.ndim > 1:
                if test_data.shape[1] > 1:
                    test_data = test_data[:, -1]

            mse = model.evaluate(test_data)
            print(f"\nTest MSE: {mse}")

    except Exception as e:
        print(f"Error during model training: {e}")

    # Calculate and print training time
    current_time = time.process_time()
    training_time = current_time - start_time
    print(f"Total Training Time: {training_time} seconds")

    return model

## Training the ARIMA model

In [25]:
import os

# Define the directory name
model_dir = "models"

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Directory '{model_dir}' created successfully.")
else:
    print(f"Directory '{model_dir}' already exists.")

# Training the ARIMA model
# seq_len = 90  # (timestamps)
p = 5  # AR order
d = 1  # Difference order
q = 1  # MA order
print_every = 100

# Assuming you have some time series data
# For ARIMA, we need to prepare a univariate time series
# If your data is already in train_loader, you need to extract it

# Example: Extract data from your existing loader
# First, get a batch sample to understand structure
sample_batch = next(iter(train_loader))
sample_features = sample_batch[0].numpy()  # Shape: [batch_size, seq_len, n_features]
sample_targets = sample_batch[1].numpy()   # Shape: [batch_size, 1]

# For ARIMA, we need to flatten this to a single time series
# This is just an example approach - adjust based on your actual data
train_data_arima = []
for batch_idx in range(len(train_loader)):
    batch = next(iter(train_loader))
    features = batch[0]
    targets = batch[1]
    # Add target values to our time series
    train_data_arima.extend(targets.numpy().flatten())

# Alternative: If using synthetic/sample data for testing
# import numpy as np
# np.random.seed(42)
# train_data_arima = np.random.randn(1000)  # Example random data

# Train the ARIMA model
arima_model = train_arima(
    train_data=train_data_arima,
    order=(p, d, q),
    print_every=print_every
)

# Save the ARIMA model
with open("models/arima_model.pkl", "wb") as f:
    pickle.dump(arima_model, f)

print("ARIMA model saved to models/arima_model.pkl")

Directory 'models' already exists.
Starting Training of ARIMA model
Fitting ARIMA(5, 1, 1) model...


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Model fitting completed

Model Summary:
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:               429056
Model:                 ARIMA(5, 1, 1)   Log Likelihood              166828.258
Date:                Sun, 09 Mar 2025   AIC                        -333642.516
Time:                        09:27:42   BIC                        -333565.730
Sample:                             0   HQIC                       -333620.639
                             - 429056                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0006      0.002     -0.416      0.677      -0.004       0.002
ar.L2          0.0007      0.002      0.426      0.670      -0.002       0.004
ar.L3       

## Training the GRU model

In [27]:
# seq_len = 90  # (timestamps)
n_hidden = 256
n_layers = 2
n_epochs = 5
print_every = 100
lr = 0.001
gru_model = train(
    train_loader,
    learn_rate=lr,
    hidden_dim=n_hidden,
    n_layers=n_layers,
    n_epochs=n_epochs,
    model_type="GRU",
    print_every=print_every,
)

Starting Training of GRU model
Epoch 1 - Step: 100/419 - Average Loss for Epoch: 0.010032557991798967
Epoch 1 - Step: 200/419 - Average Loss for Epoch: 0.005623011600400787
Epoch 1 - Step: 300/419 - Average Loss for Epoch: 0.004012226227399272
Epoch 1 - Step: 400/419 - Average Loss for Epoch: 0.0031515160200069657
Epoch 1/5 Done, Total Loss: 0.0030311964292369276
Time Elapsed for Epoch: 3990.4072281649987 seconds
Epoch 2 - Step: 100/419 - Average Loss for Epoch: 0.0004712287135771476
Epoch 2 - Step: 200/419 - Average Loss for Epoch: 0.00042374114273115995
Epoch 2 - Step: 300/419 - Average Loss for Epoch: 0.0003979211723587165
Epoch 2 - Step: 400/419 - Average Loss for Epoch: 0.0003748137864386081
Epoch 2/5 Done, Total Loss: 0.00036974055392485387
Time Elapsed for Epoch: 3970.6969201939974 seconds
Epoch 3 - Step: 100/419 - Average Loss for Epoch: 0.0002609270083485171
Epoch 3 - Step: 200/419 - Average Loss for Epoch: 0.00026672345782571935
Epoch 3 - Step: 300/419 - Average Loss for Epoc

## Save the GRU model

In [28]:
torch.save(gru_model.state_dict(), "./models/gru_model.pt")

## Train and Save an LSTM model

In [ ]:
lstm_model = train(
    train_loader,
    learn_rate=lr,
    hidden_dim=n_hidden,
    n_layers=n_layers,
    n_epochs=n_epochs,
    model_type="LSTM",
    print_every=print_every,
)

Starting Training of LSTM model
Epoch 1 - Step: 100/419 - Average Loss for Epoch: 0.020150415431708098
Epoch 1 - Step: 200/419 - Average Loss for Epoch: 0.011250900059239939


In [ ]:
torch.save(lstm_model.state_dict(), "./models/lstm_model.pt")

As we can see from the training time of both models, the GRU model is the clear winner in terms of speed, as we have mentioned earlier. The GRU finished 5 training epochs faster than the LSTM model.

## Load the ARIMA model

In [ ]:
import pickle

# move device to cpu for evaluation to avoid GPU memory run
device = "cpu"

# Load the ARIMA model
try:
    with open("./models/arima_model.pkl", "rb") as f:
        arima_model = pickle.load(f)
    print("ARIMA model loaded successfully")
except FileNotFoundError:
    print("ARIMA model file not found. Please make sure the model has been trained and saved.")
except Exception as e:
    print(f"Error loading ARIMA model: {e}")

# Display model information
if 'arima_model' in locals():
    print(f"ARIMA model order: {arima_model.order}")
    print("Model summary:")
    print(arima_model.summary())

## Load the GRU model

In [ ]:
# move device to cpu for evaluation to avoid GPU memory run
device = "cpu"

In [ ]:
hidden_dim = 256
input_dim = 5
output_dim = 1
n_layers = 2
gru_model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
gru_model.load_state_dict(torch.load("./models/gru_model.pt"))

In [ ]:
# Move the model to the appropriate device
gru_model.to(device)

## Load the LSTM model

In [ ]:
hidden_dim = 256
input_dim = 5
output_dim = 1
n_layers = 2
lstm_model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
lstm_model.load_state_dict(torch.load("./models/lstm_model.pt"))

In [ ]:
# Move the model to the appropriate device
lstm_model.to(device)

## Model Evaluation



In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate Mean Absolute Percentage Error (MAPE)

    Args:
        y_true: Actual values
        y_pred: Predicted values

    Returns:
        MAPE value as percentage
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Add small constant to avoid division by zero
    epsilon = 1e-10
    # Return MAPE as percentage
    return 100 * np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + epsilon)))

def evaluate_model(model_name, y_true, y_pred):
    """
    Calculate and display various evaluation metrics for a model

    Args:
        model_name: Name of the model
        y_true: Actual values
        y_pred: Predicted values

    Returns:
        Dictionary of evaluation metrics
    """
    # Calculate metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    # Store metrics in dictionary
    metrics = {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE (%)': mape
    }

    # Print metrics in a formatted way
    print(f"\n----- {model_name} Model Evaluation -----")
    print(f"Mean Squared Error (MSE): {mse:.6f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
    print(f"Mean Absolute Error (MAE): {mae:.6f}")
    print(f"R² Score: {r2:.6f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    return metrics

def evaluate_gru_model(test_loader, gru_model, device='cpu'):
    """
    Evaluate the GRU model on test data

    Args:
        test_loader: DataLoader containing test data
        gru_model: Trained GRU model
        device: Device to run evaluation on

    Returns:
        Evaluation metrics
    """
    gru_model.eval()  # Set model to evaluation mode
    y_true_list = []
    y_pred_list = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device).float()
            targets = targets.to(device).float()

            # Initialize hidden state
            h = gru_model.init_hidden(inputs.size(0))
            h = h.data

            # Forward pass
            outputs, _ = gru_model(inputs, h)

            # Store predictions and actual values
            y_true_list.extend(targets.cpu().numpy().flatten())
            y_pred_list.extend(outputs.cpu().numpy().flatten())

    # Evaluate the model
    metrics = evaluate_model('GRU', y_true_list, y_pred_list)

    return metrics, y_true_list, y_pred_list

def evaluate_lstm_model(test_loader, lstm_model, device='cpu'):
    """
    Evaluate the LSTM model on test data

    Args:
        test_loader: DataLoader containing test data
        lstm_model: Trained LSTM model
        device: Device to run evaluation on

    Returns:
        Evaluation metrics
    """
    lstm_model.eval()  # Set model to evaluation mode
    y_true_list = []
    y_pred_list = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device).float()
            targets = targets.to(device).float()

            # Initialize hidden state
            h = lstm_model.init_hidden(inputs.size(0))
            h = tuple([e.data for e in h])

            # Forward pass
            outputs, _ = lstm_model(inputs, h)

            # Store predictions and actual values
            y_true_list.extend(targets.cpu().numpy().flatten())
            y_pred_list.extend(outputs.cpu().numpy().flatten())

    # Evaluate the model
    metrics = evaluate_model('LSTM', y_true_list, y_pred_list)

    return metrics, y_true_list, y_pred_list

def evaluate_arima_model(test_data, arima_model):
    """
    Evaluate the ARIMA model on test data

    Args:
        test_data: Test data (array or Series)
        arima_model: Trained ARIMA model

    Returns:
        Evaluation metrics
    """
    # Ensure test_data is in the right format
    if isinstance(test_data, torch.Tensor):
        test_data = test_data.numpy()

    # Make predictions
    predictions = arima_model.predict(steps=len(test_data))

    # Evaluate the model
    metrics = evaluate_model('ARIMA', test_data, predictions)

    return metrics, test_data, predictions

def compare_models(metrics_list):
    """
    Compare different models based on their evaluation metrics

    Args:
        metrics_list: List of dictionaries containing evaluation metrics for different models
    """
    # Create DataFrame from metrics
    df = pd.DataFrame(metrics_list)

    # Set Model as index for better display
    df = df.set_index('Model')

    # Display the comparison table
    print("\n----- Model Comparison -----")
    print(df)

    # Create bar chart for MSE comparison
    plt.figure(figsize=(12, 6))

    # Plot MSE
    plt.subplot(2, 2, 1)
    plt.bar(df.index, df['MSE'])
    plt.title('Mean Squared Error (MSE)')
    plt.ylabel('MSE')

    # Plot MAE
    plt.subplot(2, 2, 2)
    plt.bar(df.index, df['MAE'])
    plt.title('Mean Absolute Error (MAE)')
    plt.ylabel('MAE')

    # Plot R²
    plt.subplot(2, 2, 3)
    plt.bar(df.index, df['R²'])
    plt.title('R² Score')
    plt.ylabel('R²')

    # Plot MAPE
    plt.subplot(2, 2, 4)
    plt.bar(df.index, df['MAPE (%)'])
    plt.title('Mean Absolute Percentage Error (MAPE)')
    plt.ylabel('MAPE (%)')

    plt.tight_layout()
    plt.show()

    # Determine the best model for each metric
    best_mse = df['MSE'].idxmin()
    best_mae = df['MAE'].idxmin()
    best_r2 = df['R²'].idxmax()
    best_mape = df['MAPE (%)'].idxmin()

    print("\n----- Best Models -----")
    print(f"Best model according to MSE: {best_mse}")
    print(f"Best model according to MAE: {best_mae}")
    print(f"Best model according to R²: {best_r2}")
    print(f"Best model according to MAPE: {best_mape}")

## Evaluate performance of ARIMA

In [ ]:
arima_metrics, arima_true, arima_pred = evaluate_arima_model(test_data_arima, arima_model)

## Evaluate performance of GRU

In [ ]:
gru_metrics, gru_true, gru_pred = evaluate_gru_model(test_loader, gru_model, device)

## Evaluate performance of LSTM

In [ ]:
lstm_metrics, lstm_true, lstm_pred = evaluate_lstm_model(test_loader, lstm_model, device)

Comparing all the 3 models

In [ ]:
all_metrics = [gru_metrics, lstm_metrics, arima_metrics]
compare_models(all_metrics)

# Some visualizations

Lastly, let's do some visualizations on random sets of our predicted output vs the actual consumption data for some states.

In [ ]:
states_list = list(test_x.keys())

In [ ]:
states_list

In [ ]:
# Plot predictions vs actual values for each model
plt.figure(figsize=(15, 10))

# Plot for GRU
plt.subplot(3, 1, 1)
plt.plot(gru_true[:100], label='Actual')
plt.plot(gru_pred[:100], label='Predicted')
plt.title('GRU: Actual vs Predicted')
plt.legend()

# Plot for LSTM
plt.subplot(3, 1, 2)
plt.plot(lstm_true[:100], label='Actual')
plt.plot(lstm_pred[:100], label='Predicted')
plt.title('LSTM: Actual vs Predicted')
plt.legend()

# Plot for ARIMA
plt.subplot(3, 1, 3)
plt.plot(arima_true[:100], label='Actual')
plt.plot(arima_pred[:100], label='Predicted')
plt.title('ARIMA: Actual vs Predicted')
plt.legend()

plt.tight_layout()
plt.show()

Looks like the models are largely successful in predicting the trends of energy consumption. While they may still get some changes wrong, such as delays in predicting a drop in consumption, the predictions follow very closely to the actual line on the test set. This is due to the nature of energy consumption data and the fact that there are patterns and cyclical changes that the model can account for. Tougher time-series prediction problems such as stock price prediction or sales volume prediction may have data that is largely random or doesn’t have predictable patterns, and in such cases, the accuracy will definitely be lower.

## What's next?

* Use more data.
* Use more complex (more layers) networks.

Referenced from https://blog.floydhub.com/gru-with-pytorch/